<h2>The RSA crypto scheme, sometimes referred to as the
Rivest–Shamir–Adleman algorithm, is currently the
most widely used asymmetric cryptographic scheme</h2>
<h3> There are many applications for RSA, but in practice it
is most often used for:</h3><br />
1- encryption of small pieces of data, especially for key
transport <br />
2- digital signatures

<br /><h3>Here we are going to implement RSA with public and private keys up to 512 bits within limit of time for setup and decryption less than 3 seconds!<h3>

First let us import our neede libraries

In [1]:
import random
import numpy as np
from random import randrange, randint
import timeit

1- We need to implement Euclid's algorithm for determining the greatest common divisor
Use iteration to make it faster for larger integers

In [2]:
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

2- Euclid's extended algorithm for finding the multiplicative inverse of two numbers

In [3]:
def multiplicative_inverse(e, phi):
    d = 0
    x1 = 0
    x2 = 1
    y1 = 1
    temp_phi = phi
    
    while e > 0:
        temp1 = temp_phi/e
        temp2 = temp_phi - temp1 * e
        temp_phi = e
        e = temp2
        
        x = x2- temp1* x1
        y = d - temp1 * y1
        
        x2 = x1
        x1 = x
        d = y1
        y1 = y
    
    if temp_phi == 1:
        return d + phi

3- Tests to see if a number is prime. and to keep our time limit we need to use a trick to make our test faster so we implement Miller-rabin for primality test 

In [4]:
def is_prime(n, k=10):
    if n == 2:
        return True
    if not n & 1:
        return False

    def check(a, s, d, n):
        x = pow(a, d, n)
        if x == 1:
            return True
        for i in xrange(s - 1):
            if x == n - 1:
                return True
            x = pow(x, 2, n)
        return x == n - 1

    s = 0
    d = n - 1

    while d % 2 == 0:
        d >>= 1
        s += 1

    for i in xrange(k):
        a = randrange(2, n - 1)
        if not check(a, s, d, n):
            return False
    return True

4- here we are going to generat prime number according to the size entered bu the user

In [5]:
def generate_big_prime(n, p_in=0):
    found_prime = False
    while not found_prime:
        p = randint(2**(n-1), 2**n)
        if is_prime(p, 100) and p != p_in:
            return p

5- generate public and private key pair 

In [6]:
def generate_keypair(p, q, n):
    if not (is_prime(p) and is_prime(q)):
        raise ValueError('Both numbers must be prime.')
    elif p == q:
        raise ValueError('p and q cannot be equal')
    n = np.multiply(p , q)
    phi = np.multiply((p-1) , (q-1))
    e = random.randrange(1, phi)
    # Use Euclid's Algorithm to verify that e and phi(n) are comprime
    g = gcd(e, phi)
    while g != 1:
        e = random.randrange(1, phi)
        g = gcd(e, phi)
    # Use Extended Euclid's Algorithm to generate the private key
    d = multiplicative_inverse(e, phi)
    
    # Return public and private keypair
    # Public key is (e, n) and private key is (d, n)
    return ((e, n), (d, n))

6- here we are gonna finish we need Encryption and Decryption methods of our RSA

In [7]:
def encrypt(pk, plaintext):
    #Unpack the key into it's components
    key, n = pk
    cipher = [(pow(ord(char), key , n)) for char in plaintext]
    return cipher

In [8]:
def decrypt(pk, ciphertext):
    #Unpack the key into its components
    key, n = pk
    #Generate the plaintext based on the ciphertext and key using a^b mod m
    plain = [chr(pow(char , key , n)) for char in ciphertext]
    #Return the array of bytes as a string
    return ''.join(plain)

Finally our main method to call previous methods 

In [9]:
    print "RSA Encrypter/ Decrypter"

RSA Encrypter/ Decrypter


In [10]:
    p_size =int(raw_input("Enter size of P in bits : "))
    q_size =int(raw_input("Enter size of q in bits : "))

Enter size of P in bits : 512
Enter size of q in bits : 512


In [11]:
    setup_start = timeit.default_timer()
    p = generate_big_prime(p_size)
    q = generate_big_prime(q_size, p)  

In [12]:
    print('Value of P : ' + str(p))
    print('Value of q : ' + str(q))

Value of P : 12087358972109044312974199780610576143859224860392927657426457966944737118766853181783230227246199334029444727894602466604031917029249733464392512697264367
Value of q : 12787775935841985438418206507386951665949370182060895044603181736439518021901943001228228447459978528469753545060160414177292232815948504456802379177374489


In [13]:
    print "Generating your public/private keypairs now . . ."
    public, private = generate_keypair(p, q, p_size)
    print "Your public key is ", public ," and your private key is ", private

Generating your public/private keypairs now . . .
Your public key is  (94496442079492813285355380106740904978162008228234018132031234838567139111838770036373828556584530894643486038142142710953657604114887573401611442719702884923042759394396365609840995154126289563028135300465233731846511696036663227899909335970573223047659559223628992585437984915546070390263891246533042907397L, 154570438191419753304749555869253253440327809877226095066778834558817217921575340702411569698874407600471604802365720638280226439349189100078561035722518096755123515932108393693330766520862191024819903349149589103858653737890449234756562769690573231616167190413370411945703378034990262687859272632371794533463L)  and your private key is  (164181138760238417724873549143199284960610303014945274095443607781088520394083663411820270212886744346110478754043508140149599671427046650866310840132855286788563485780497308980720463272056452806902154871969048408050202045639377343468469896301387317137943468345621764857459133941

In [14]:
setup_end = timeit.default_timer()
print("setup time is : " + str((setup_end - setup_start)))

setup time is : 0.899255990982


In [15]:
    message = raw_input("Enter a message to encrypt with public key: ")
    encrypted_msg = encrypt(public, message)
    print "Your encrypted message is: "
    print ''.join(map(lambda x: str(x), encrypted_msg))
    print "Decrypting message with private key  . . ."
    

Enter a message to encrypt with public key: hi saddam how are you that's great job man!
Your encrypted message is: 
40402104748841152391853491823133361100071431412740351039538948085833490968428441629506432905099243989560173781672735453961997945221023922749986128267384341189604154591719959682037459244426106004512938030118908420300991940535121119061717683837010239139340161279970337966194661032230431470567926802545458560253428525314900011568212624252079508814390384266310220880142116725116314033852282920935355881354695539273472734163603183392133169001985730329464122588236799335606435716755391667679747023194759532936633422929863278474924468318393577944920895552731982431412749641649662493545366081398972720550306840972046591743683218761092137386010497191384980242182452254795090673782846518597027891308506936667654521671769679803062844355173677119713006013462904322961757275105306447582713858142451560095513773098003589143367772679113344538080508575766998238591226502105007888073664388270828570926

In [16]:
    decrypt_start = timeit.default_timer()
    decrypted_message =  decrypt(private, encrypted_msg)
    decrypt_end = timeit.default_timer()  

In [17]:
    print "Your original massage is: " + str(decrypted_message)
    print("Decryption time is : " + str((decrypt_end - decrypt_start)))

Your original massage is: hi saddam how are you that's great job man!
Decryption time is : 0.280925989151


In [18]:
    print("setup Time + Decryption Time = " + str((decrypt_end - decrypt_start) + (setup_end - setup_start)))

setup Time + Decryption Time = 1.18018198013
